# 5-6: Control & Recovery - Advanced Flow and Resilience for Azure OpenAI Agents

**Original Code by Dave Ebbelaar** - Extended for Azure OpenAI by Arturo Quiroga

This notebook demonstrates two advanced building blocks for Azure OpenAI agents:
- **Control**: Deterministic decision-making, intent classification, routing, and process orchestration
- **Recovery**: Robust error handling, retry logic, circuit breaker, and resilience patterns

These components enable agents to manage complex workflows, route requests, and recover gracefully from failures.

## Prerequisites

Ensure you have the following environment variables set:
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY` (or use Managed Identity)
- `AZURE_OPENAI_GPT4_DEPLOYMENT`

Install required packages:
- openai
- azure-identity
- python-dotenv
- pydantic

In [1]:
# Install required packages
#!pip install openai azure-identity python-dotenv pydantic

In [2]:
# Import required libraries and set up environment
import os
import time
import json
import logging
from typing import Optional, Any, Callable, List, Literal
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
from pydantic import BaseModel, Field, ValidationError

# Load environment variables
load_dotenv()

# Configure logging for recovery components
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Azure OpenAI Client Initialization

This function initializes the Azure OpenAI client using either API key or Managed Identity, following Azure best practices for authentication.

In [3]:
def get_azure_openai_client():
    """Initialize Azure OpenAI client with proper authentication."""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
    
    if not endpoint:
        raise ValueError("AZURE_OPENAI_ENDPOINT environment variable is required")
    
    if api_key:
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
        )
    else:
        credential = DefaultAzureCredential()
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=credential,
            api_version=api_version,
        )
    
    return client

# Initialize client
client = get_azure_openai_client()
deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
print(f"Azure OpenAI client initialized with deployment: {deployment_name}")

Azure OpenAI client initialized with deployment: gpt-4.1


# 1. Intent Classification Implementation

We use a Pydantic schema to classify user input into one of several intent categories, with confidence and reasoning. This enables downstream routing and control logic.

In [4]:
class IntentClassification(BaseModel):
    """
    Schema for intent classification results.
    """
    intent: Literal["question", "request", "complaint", "greeting", "goodbye"] = Field(
        description="The classified intent of the user input"
    )
    confidence: float = Field(
        ge=0.0, le=1.0, 
        description="Confidence score between 0 and 1"
    )
    reasoning: str = Field(
        description="Explanation for the classification decision"
    )
    entities: List[str] = Field(
        default_factory=list,
        description="Key entities extracted from the input"
    )

def classify_intent(user_input: str) -> IntentClassification:
    """
    Classify user input into predefined intent categories using Azure OpenAI.
    """
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    
    system_prompt = """
    Classify the user input into one of these intents: question, request, complaint, greeting, goodbye.
    Return JSON in this format:
    {"intent": "...", "confidence": 0.95, "reasoning": "...", "entities": ["..."]}
    """
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input},
            ],
            temperature=0.1,
            max_tokens=800,
            response_format={"type": "json_object"}
        )
        response_text = response.choices[0].message.content
        json_data = json.loads(response_text)
        return IntentClassification(**json_data)
    except Exception as e:
        print(f"Intent classification error: {e}")
        return IntentClassification(
            intent="question",
            confidence=0.0,
            reasoning=f"Error in classification: {str(e)}",
            entities=[]
        )

# 2. Request Routing System

This schema and function route requests to the appropriate department and assign a priority, based on the classified intent and content.

In [5]:
class RoutingDecision(BaseModel):
    """
    Schema for routing decisions.
    """
    department: Literal["sales", "support", "billing", "technical", "general"] = Field(
        description="Department to route the request to"
    )
    priority: Literal["low", "medium", "high", "urgent"] = Field(
        description="Priority level of the request"
    )
    requires_human: bool = Field(
        description="Whether human intervention is required"
    )
    estimated_resolution_time: str = Field(
        description="Estimated time to resolve (e.g., '5 minutes', '2 hours', '1 day')"
    )

def route_request(user_input: str, intent: IntentClassification) -> RoutingDecision:
    """
    Route requests to appropriate departments based on content and intent.
    """
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    
    system_prompt = f"""
    Route this request to the appropriate department based on the content and intent.
    User intent: {intent.intent}
    User input: {user_input}
    Return JSON in this format:
    {{"department": "...", "priority": "...", "requires_human": true, "estimated_resolution_time": "..."}}
    """
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Route this: {user_input}"},
            ],
            temperature=0.1,
            max_tokens=500,
            response_format={"type": "json_object"}
        )
        response_text = response.choices[0].message.content
        json_data = json.loads(response_text)
        return RoutingDecision(**json_data)
    except Exception as e:
        print(f"Routing error: {e}")
        return RoutingDecision(
            department="general",
            priority="medium",
            requires_human=True,
            estimated_resolution_time="30 minutes"
        )

# 3. Control Agent with Decision Trees

This agent manages conversation flow, maintains history, and routes requests through proper decision trees for robust control.

In [6]:
def handle_greeting(user_input: str) -> str:
    return "Hello! How can I assist you today? I can help with questions, requests, or connect you with the right department."

def handle_goodbye(user_input: str) -> str:
    return "Thank you for contacting us! Have a great day and feel free to reach out if you need anything else."

def answer_question(question: str, routing: RoutingDecision) -> str:
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    department_context = {
        "sales": "You are a sales representative. Focus on how our products can solve their needs.",
        "support": "You are a support agent. Provide helpful troubleshooting and guidance.",
        "billing": "You are a billing specialist. Help with account and payment questions.",
        "technical": "You are a technical expert. Provide detailed technical information.",
        "general": "You are a general assistant. Provide helpful information and guidance."
    }
    context = department_context.get(routing.department, department_context["general"])
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": question},
            ],
            temperature=0.7,
            max_tokens=1000,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"I'm having trouble processing your question right now. Please try again or contact our {routing.department} team directly."

def process_request(request: str, routing: RoutingDecision) -> str:
    if routing.requires_human:
        return f"I've routed your request to our {routing.department} team (Priority: {routing.priority}). Expected response time: {routing.estimated_resolution_time}. Someone will get back to you soon!"
    else:
        return f"Processing your request: {request}. This has been categorized as {routing.priority} priority for the {routing.department} team."

def handle_complaint(complaint: str, routing: RoutingDecision) -> str:
    return f"I understand your concern and I'm sorry you're experiencing this issue. I've escalated this to our {routing.department} team with {routing.priority} priority. Expected resolution time: {routing.estimated_resolution_time}. A specialist will contact you shortly to resolve this matter."

def route_based_on_intent(user_input: str) -> tuple[str, IntentClassification, RoutingDecision]:
    classification = classify_intent(user_input)
    intent = classification.intent
    routing = route_request(user_input, classification)
    if intent == "greeting":
        result = handle_greeting(user_input)
    elif intent == "goodbye":
        result = handle_goodbye(user_input)
    elif intent == "question":
        result = answer_question(user_input, routing)
    elif intent == "request":
        result = process_request(user_input, routing)
    elif intent == "complaint":
        result = handle_complaint(user_input, routing)
    else:
        result = "I'm not sure how to help with that. Let me connect you with a human agent."
    return result, classification, routing

class ControlAgent:
    """
    Advanced control agent that manages complex decision trees and workflows.
    """
    def __init__(self):
        self.client = get_azure_openai_client()
        self.deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
        self.conversation_history = []
    def process_input(self, user_input: str) -> dict:
        response, classification, routing = route_based_on_intent(user_input)
        self.conversation_history.append({
            "user_input": user_input,
            "classification": classification.model_dump(),
            "routing": routing.model_dump(),
            "response": response
        })
        return {
            "response": response,
            "classification": classification.model_dump(),
            "routing": routing.model_dump(),
            "conversation_turn": len(self.conversation_history)
        }
    def get_conversation_summary(self) -> str:
        if not self.conversation_history:
            return "No conversation history yet."
        intents = [turn["classification"]["intent"] for turn in self.conversation_history]
        departments = [turn["routing"]["department"] for turn in self.conversation_history]
        return f"Conversation summary: {len(self.conversation_history)} turns, Intents: {list(set(intents))}, Departments involved: {list(set(departments))}"

# 4. Retry Logic and Exponential Backoff

This section implements robust retry logic with exponential backoff and jitter, following Azure best practices for reliability.

In [7]:
class RetryConfig:
    """Configuration for retry logic."""
    def __init__(
        self,
        max_attempts: int = 3,
        initial_delay: float = 1.0,
        max_delay: float = 60.0,
        exponential_base: float = 2.0,
        jitter: bool = True
    ):
        self.max_attempts = max_attempts
        self.initial_delay = initial_delay
        self.max_delay = max_delay
        self.exponential_base = exponential_base
        self.jitter = jitter

def exponential_backoff(attempt: int, config: RetryConfig) -> float:
    delay = config.initial_delay * (config.exponential_base ** attempt)
    delay = min(delay, config.max_delay)
    if config.jitter:
        import random
        delay *= (0.5 + random.random() * 0.5)
    return delay

def retry_with_backoff(
    func: Callable,
    config: RetryConfig = None,
    exceptions: tuple = None
) -> Any:
    if config is None:
        config = RetryConfig()
    if exceptions is None:
        exceptions = (Exception,)
    last_exception = None
    for attempt in range(config.max_attempts):
        try:
            return func()
        except exceptions as e:
            last_exception = e
            if attempt < config.max_attempts - 1:
                delay = exponential_backoff(attempt, config)
                logger.warning(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay:.2f} seconds...")
                time.sleep(delay)
            else:
                logger.error(f"All {config.max_attempts} attempts failed. Last error: {e}")
    raise last_exception

# 5. Resilient Intelligence with Error Handling

This function demonstrates resilient intelligence with comprehensive error handling and fallback responses for Azure OpenAI calls.

In [8]:
def resilient_intelligence(prompt: str) -> str:
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    def make_request():
        try:
            response = client.chat.completions.create(
                model=deployment_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=1000,
                timeout=30
            )
            return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Azure OpenAI request failed: {e}")
            raise
    try:
        config = RetryConfig(max_attempts=3, initial_delay=1.0)
        return retry_with_backoff(make_request, config)
    except Exception as e:
        logger.error(f"All attempts failed: {e}")
        return f"I'm experiencing technical difficulties right now. Please try again later. Error: {str(e)[:100]}"

# 6. User Information Extraction with Fallbacks

This function extracts user information with graceful degradation, partial data handling, and multiple fallback strategies.

In [9]:
class UserInfo(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    age: Optional[int] = None
    phone: Optional[str] = None
    company: Optional[str] = None

def extract_user_info_with_fallback(prompt: str) -> dict:
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    system_prompt = """
    Extract user information from the text and return as JSON.
    {"name": "string or null", "email": "string or null", "age": "number or null", "phone": "string or null", "company": "string or null"}
    Use null for missing fields.
    """
    def extract_attempt():
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            temperature=0.0,
            max_tokens=500,
            response_format={"type": "json_object"}
        )
        response_text = response.choices[0].message.content
        json_data = json.loads(response_text)
        return UserInfo(**json_data)
    try:
        user_data = retry_with_backoff(
            extract_attempt,
            RetryConfig(max_attempts=2),
            (json.JSONDecodeError, ValidationError, Exception)
        )
        return {
            "success": True,
            "data": user_data.model_dump(),
            "source": "structured_extraction"
        }
    except ValidationError as e:
        logger.warning(f"Validation failed, attempting partial extraction: {e}")
        try:
            simple_prompt = f"Extract any available information from this text as JSON: {prompt}\nUse this format: {{'name': 'string or null', 'email': 'string or null'}}"
            response = client.chat.completions.create(
                model=deployment_name,
                messages=[{"role": "user", "content": simple_prompt}],
                temperature=0.0,
                max_tokens=300,
                response_format={"type": "json_object"}
            )
            partial_data = json.loads(response.choices[0].message.content)
            user_info = UserInfo(
                name=partial_data.get("name"),
                email=partial_data.get("email")
            )
            return {
                "success": True,
                "data": user_info.model_dump(),
                "source": "partial_extraction",
                "warning": "Some fields could not be extracted"
            }
        except Exception as fallback_error:
            logger.error(f"Fallback extraction also failed: {fallback_error}")
        return {
            "success": False,
            "data": {"name": None, "email": None, "age": None, "phone": None, "company": None},
            "source": "fallback",
            "error": str(e)
        }
    except Exception as e:
        logger.error(f"Complete extraction failure: {e}")
        return {
            "success": False,
            "data": {"name": None, "email": None, "age": None, "phone": None, "company": None},
            "source": "error_fallback",
            "error": str(e)
        }

# 7. Circuit Breaker Pattern Implementation

This agent implements a circuit breaker to prevent cascading failures and monitor service health.

In [10]:
class ResilientAgent:
    """
    Agent with comprehensive resilience patterns and error recovery.
    """
    def __init__(self, retry_config: RetryConfig = None):
        self.client = get_azure_openai_client()
        self.deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
        self.retry_config = retry_config or RetryConfig()
        self.error_count = 0
        self.success_count = 0
        self.circuit_breaker_threshold = 5  # Open circuit after 5 consecutive failures
        self.circuit_open = False
        self.last_success_time = time.time()
    def _should_circuit_break(self) -> bool:
        if self.error_count >= self.circuit_breaker_threshold:
            if time.time() - self.last_success_time > 300:
                return True
        return False
    def _reset_circuit(self):
        self.circuit_open = False
        self.error_count = 0
        self.last_success_time = time.time()
    def safe_chat(self, prompt: str, fallback_response: str = None) -> dict:
        if self.circuit_open or self._should_circuit_break():
            self.circuit_open = True
            fallback = fallback_response or "Service temporarily unavailable. Please try again later."
            return {
                "success": False,
                "response": fallback,
                "source": "circuit_breaker",
                "error": "Circuit breaker open"
            }
        def make_chat_request():
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=1000,
                timeout=30
            )
            return response.choices[0].message.content
        try:
            response = retry_with_backoff(make_chat_request, self.retry_config)
            self.success_count += 1
            self._reset_circuit()
            return {
                "success": True,
                "response": response,
                "source": "azure_openai",
                "attempts": "multiple" if self.retry_config.max_attempts > 1 else "single"
            }
        except Exception as e:
            self.error_count += 1
            logger.error(f"Chat request failed after retries: {e}")
            fallback = fallback_response or f"I'm having technical difficulties. Please try again. Error: {str(e)[:50]}"
            return {
                "success": False,
                "response": fallback,
                "source": "error_fallback",
                "error": str(e),
                "error_count": self.error_count
            }
    def get_health_status(self) -> dict:
        return {
            "circuit_open": self.circuit_open,
            "error_count": self.error_count,
            "success_count": self.success_count,
            "last_success_time": self.last_success_time,
            "health": "healthy" if not self.circuit_open and self.error_count < 3 else "degraded"
        }

# 8. Comprehensive Testing Scenarios

Let's test the control and recovery components with various scenarios, including normal operations and error conditions.

In [11]:
# Test ControlAgent with different types of inputs
print("=== ControlAgent Demo ===\n")
test_inputs = [
    "Hello there!",
    "What is machine learning and how does it work?",
    "Please schedule a meeting for tomorrow at 2 PM",
    "I'm unhappy with the service quality, my account is not working",
    "Thanks for your help, goodbye!",
    "My API integration is failing with error 500",
    "How much does your premium plan cost?",
    "I can't log into my account and need help immediately"
]
for i, user_input in enumerate(test_inputs, 1):
    print(f"{i}. Input: '{user_input}'")
    try:
        result, classification, routing = route_based_on_intent(user_input)
        print(f"   Intent: {classification.intent} (confidence: {classification.confidence:.2f})")
        print(f"   Reasoning: {classification.reasoning}")
        print(f"   Department: {routing.department}, Priority: {routing.priority}")
        print(f"   Requires Human: {routing.requires_human}")
        print(f"   Response: {result[:100]}{'...' if len(result) > 100 else ''}")
        print()
    except Exception as e:
        print(f"   Error: {e}")
        print()

# Test the ControlAgent conversation
print("=== Testing ControlAgent Conversation ===\n")
agent = ControlAgent()
test_conversation = [
    "Hi, I need help with my account",
    "I can't access my dashboard and it's urgent",
    "Thank you for your help"
]
for msg in test_conversation:
    result = agent.process_input(msg)
    print(f"User: {msg}")
    print(f"Agent: {result['response'][:100]}{'...' if len(result['response']) > 100 else ''}")
    print(f"Classification: {result['classification']['intent']}")
    print(f"Routing: {result['routing']['department']} ({result['routing']['priority']})")
    print()
print(agent.get_conversation_summary())

# Test Recovery: Resilient Intelligence
print("\n=== Recovery: Resilient Intelligence ===")
try:
    result = resilient_intelligence("Tell me about machine learning in one sentence")
    print(f"Success: {result[:100]}...")
except Exception as e:
    print(f"Failed: {e}")

# Test Recovery: User info extraction with fallback
print("\n=== Recovery: User Info Extraction with Fallback ===")
test_prompts = [
    "My name is John Smith and my email is john@example.com",
    "Hi, I'm Jane",
    "Contact me at invalid-email-format",
]
for i, prompt in enumerate(test_prompts, 1):
    print(f"   Test {i}: '{prompt}'")
    result = extract_user_info_with_fallback(prompt)
    print(f"   Result: {result}")
    print()

# Test Recovery: Resilient agent with circuit breaker
print("\n=== Recovery: Resilient Agent with Circuit Breaker ===")
agent = ResilientAgent(RetryConfig(max_attempts=2, initial_delay=0.5))
result1 = agent.safe_chat("What is 2+2?")
print(f"Chat 1: {result1}")
result2 = agent.safe_chat(
    "Explain quantum computing", 
    fallback_response="Quantum computing is complex. Please consult our documentation."
)
print(f"Chat 2: {result2}")
health = agent.get_health_status()
print(f"Agent health: {health}")
# Simulate error scenario
print(f"\nTesting error scenario...")
original_deployment = agent.deployment_name
agent.deployment_name = "invalid-deployment-name"
result3 = agent.safe_chat("This should fail gracefully")
print(f"Error test result: {result3}")
agent.deployment_name = original_deployment
final_health = agent.get_health_status()
print(f"Final health status: {final_health}")

=== ControlAgent Demo ===

1. Input: 'Hello there!'


2025-07-29 10:57:57,788 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-07-29 10:58:00,140 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-07-29 10:58:00,140 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Routing error: 1 validation error for RoutingDecision
department
  Input should be 'sales', 'support', 'billing', 'technical' or 'general' [type=literal_error, input_value='Customer Service', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
   Intent: greeting (confidence: 0.99)
   Reasoning: The user is initiating the conversation with a friendly salutation, which is a typical greeting.
   Department: general, Priority: medium
   Requires Human: True
   Response: Hello! How can I assist you today? I can help with questions, requests, or connect you with the righ...

2. Input: 'What is machine learning and how does it work?'


2025-07-29 10:58:01,485 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-07-29 10:58:02,667 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-07-29 10:58:02,667 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Routing error: 1 validation error for RoutingDecision
department
  Input should be 'sales', 'support', 'billing', 'technical' or 'general' [type=literal_error, input_value='Technical Support - AI/ML', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error


...

Chat 1: {'success': True, 'response': '2 + 2 = **4**', 'source': 'azure_openai', 'attempts': 'multiple'}


2025-07-29 10:58:48,566 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-07-29 10:58:48,712 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/invalid-deployment-name/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 404 DeploymentNotFound"
2025-07-29 10:58:48,713 - WARNING - Attempt 1 failed: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}. Retrying in 0.44 seconds...
2025-07-29 10:58:48,712 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/invalid-deployment-name/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 404 DeploymentNotFound"
2025-07-29 10:58:48,713 - WARNING - 

Chat 2: {'success': True, 'response': "Absolutely! Here’s a clear explanation of **quantum computing**:\n\n---\n\n### What is Quantum Computing?\n\n**Quantum computing** is a branch of computing that uses the principles of quantum mechanics—the science that explains how things work at the smallest scales, like atoms and subatomic particles—to process information in entirely new ways.\n\n#### Classical vs. Quantum Computers\n\n- **Classical Computers:**  \n  The computers we use every day store and process information as **bits**, which can be either 0 or 1.\n- **Quantum Computers:**  \n  These use **quantum bits** or **qubits**, which can be 0, 1, or **both at the same time** (thanks to a property called **superposition**).\n\n---\n\n### Key Quantum Concepts\n\n1. **Superposition:**  \n   A qubit can be in a state of 0, 1, or any combination of both at once. This allows quantum computers to process many possibilities simultaneously.\n\n2. **Entanglement:**  \n   Qubits can be linked to

2025-07-29 10:58:49,298 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/invalid-deployment-name/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 404 DeploymentNotFound"
2025-07-29 10:58:49,299 - ERROR - All 2 attempts failed. Last error: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
2025-07-29 10:58:49,300 - ERROR - Chat request failed after retries: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}


Error test result: {'success': False, 'response': "I'm having technical difficulties. Please try again. Error: Error code: 404 - {'error': {'code': 'DeploymentNo", 'source': 'error_fallback', 'error': "Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}", 'error_count': 1}
Final health status: {'circuit_open': False, 'error_count': 1, 'success_count': 2, 'last_success_time': 1753801128.5691452, 'health': 'healthy'}


# Summary

In this notebook, we've implemented advanced control and recovery building blocks for Azure OpenAI agents:
- **Control**: Intent classification, request routing, and decision tree orchestration
- **Recovery**: Retry logic, exponential backoff, error handling, circuit breaker, and resilience patterns

These patterns enable robust, production-ready AI agents that can manage complex workflows and recover gracefully from failures.